<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Bias-variance lab

---

In this lab you will explore how bias and variance change using a dataset on college statistics.

In [1]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import patsy

from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.model_selection import cross_val_score, cross_val_predict,train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

np.random.seed(1)

### Load the data

In [2]:
college = pd.read_csv('../../../../../resource-datasets/college_stats/College.csv')
college.rename(columns={'Unnamed: 0':'College'}, inplace=True)
college.head(3)

,College,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Abilene Christian University,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Adelphi University,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Adrian College,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54


### Check the names of the variables and the data types contained in the college data

In [3]:
college.dtypes

College         object
Private         object
Apps             int64
Accept           int64
Enroll           int64
Top10perc        int64
Top25perc        int64
F.Undergrad      int64
P.Undergrad      int64
Outstate         int64
Room.Board       int64
Books            int64
Personal         int64
PhD              int64
Terminal         int64
S.F.Ratio      float64
perc.alumni      int64
Expend           int64
Grad.Rate        int64
dtype: object

### Clean the column names (replace "." by "_" and transform to lower case)

In [4]:
college.columns = [col.lower().replace('.','_') for col in college.columns]

### Transform the variable "Private" into 1s and 0s rather than yes/no

In [5]:
college.private = college.private.map(lambda x: 1 if x == 'Yes' else 0)

### Choose "grad_rate" as target variable

In [6]:
y = college['grad_rate'].values

### Create a feature matrix containing all variables except "grad_rate" and the college names

In [7]:
X = college.iloc[:,1:-1]
X.head(2)

,private,apps,accept,enroll,top10perc,top25perc,f_undergrad,p_undergrad,outstate,room_board,books,personal,phd,terminal,s_f_ratio,perc_alumni,expend
0,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041
1,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527


### Use the standard scaler to rescale your features and response

Hint: to rescale the response variable, you will first have to bring it intro 2D-array form and later to retransform it into 1D-array form.

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X = pd.DataFrame(X_scaled,columns=X.columns)
X.head()

,private,apps,accept,enroll,top10perc,top25perc,f_undergrad,p_undergrad,outstate,room_board,books,personal,phd,terminal,s_f_ratio,perc_alumni,expend
0,0.612553,-0.346882,-0.321205,-0.063509,-0.258583,-0.191827,-0.168116,-0.209207,-0.746356,-0.964905,-0.602312,1.270045,-0.163028,-0.115729,1.013776,-0.867574,-0.501910
1,0.612553,-0.210884,-0.038703,-0.288584,-0.655656,-1.353911,-0.209788,0.244307,0.457496,1.909208,1.215880,0.235515,-2.675646,-3.378176,-0.477704,-0.544572,0.166110
2,0.612553,-0.406866,-0.376318,-0.478121,-0.315307,-0.292878,-0.549565,-0.497090,0.201305,-0.554317,-0.905344,-0.259582,-1.204845,-0.931341,-0.300749,0.585935,-0.177290
3,0.612553,-0.668261,-0.681682,-0.692427,1.840231,1.677612,-0.658079,-0.520752,0.626633,0.996791,-0.602312,-0.688173,1.185206,1.175657,-1.615274,1.151188,1.792851
4,0.612553,-0.726176,-0.764555,-0.780735,-0.655656,-0.596031,-0.711924,0.009005,-0.716508,-0.216723,1.518912,0.235515,0.204672,-0.523535,-0.553542,-1.675079,0.241803


In [10]:
y.shape = (-1,1)
y = scaler.fit_transform(y)
y.shape = (len(y),)
y[:4]

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([-0.31825194, -0.55126184, -0.66776679, -0.37650442])

### Fit a 10-fold cross-validated linear regression model for grad_rate using all other features. Evaluate the model performance using cross_val_score, obtain the r2_score and mean_squared_error for each fold and averaged over all folds.

In [11]:
linreg = LinearRegression()

lr_scores_r2 = cross_val_score(linreg, X, y, cv=10,scoring='r2')

print(lr_scores_r2)
print(np.mean(lr_scores_r2))

lr_scores_mse = - cross_val_score(linreg, X, y, cv=10,scoring='neg_mean_squared_error')

print(lr_scores_mse)
print(np.mean(lr_scores_mse))

[0.44921644 0.35875931 0.53650887 0.48549145 0.17445097 0.38775448
 0.17393223 0.43964213 0.61095627 0.3575135 ]
0.3974225630451289
[0.53143428 0.67709959 0.47839047 0.58839311 0.75469774 0.58657544
 0.64064922 0.55468051 0.36236785 0.6052721 ]
0.5779560308061675


### Create a function with the name `predict_from_samples` that will iteratively predict your target from different train-test splits

This will be used to calculate the bias and the variance afterwards.
We want to produce a range of different models fitted on a random choice of data points and making predictions for the remaining data points.


Your function should:

1. take the following arguments:
    * `model`: a regression model 
    * `X`: predictor matrix/dataframe 
    * `y`: target variable 
    * `number_of_splits`: a number indicating how many times the dataset should be split randomly into training and test sets

2. return a dataframe `yhat_tracker` containing columns for 
    * the true values of `y` (obtained from the `y` passed as an argument)
    * the predictions made for y in each of the `number_of_splits` into training/test sets

3. in the function body
    - initialize the dataframe `yhat_tracker` with a single column `y` for the true values
    - initialize a list `rowinds` containing the indices of yhat_tracker
    - create a loop over `number_of_splits`
        - within the loop, create a train/test split of rowinds
        - create training and test sets from y and X by subsetting on the indices obtained from the train/test split
        - fit the model on the training data
        - obtain predictions for those y which are currently in the test set
        - set the predicted values for those `y` which are currently in the training set to `np.nan` 
        - insert the predictions from the current loop as a new column into `yhat_tracker`
    - return yhat_tracker 
    
In the end, the returned data frame should contain one column with the true y-values and `number_of_splits` columns with predicted y-values for the different test sets. Each of the prediction columns contains only as many values as there have been in the test set each time.

In [12]:
def predict_from_samples(model, X, y, number_of_splits=100):
    
    yhat_tracker = pd.DataFrame({'ytrue':y})
    
    rowinds = list(range(X.shape[0]))
    
    for i in range(number_of_splits):
        
        train_inds, test_inds = train_test_split(rowinds, test_size=0.33)
                
        X_train, Y_train = X.iloc[train_inds, :], y[train_inds]
        X_test, Y_test = X.iloc[test_inds, :], y[test_inds]
        
        model.fit(X_train, Y_train)
        yhats = model.predict(X_test)
        
        yhat_tracker['sample'+str(i+1)] = np.nan
        yhat_tracker.iloc[test_inds, -1] = yhats
        
    return yhat_tracker

### Create different predictor datasets

To see what happens to bias and variance as the predictors change, create a few versions of X that have different numbers of predictors in them:

* model $X$ from above including all features
* a model $X_{small}$ including only one feature (e.g. personal)
* a model $X_{overfit}$ created with the patsy formula below 
    - taking the cube takes into account 
        - all original features
        - all features that could be created by squaring or cubing a single column
        - all products of two or three different columns
        - all products of the square of one column with a different column
        - -1 excludes the intercept

In [13]:
overfit_formula = '~ ('+' + '.join(X.columns)+')**3 -1'
X_overfit = patsy.dmatrix(overfit_formula, data=X, return_type='dataframe')

In [14]:
overfit_formula

'~ (private + apps + accept + enroll + top10perc + top25perc + f_undergrad + p_undergrad + outstate + room_board + books + personal + phd + terminal + s_f_ratio + perc_alumni + expend)**3 -1'

In [15]:
X_small = X[['personal']]

### Use the `predict_from_samples` function you wrote above to get the predicted values for $X$, $X_{small}$, $X_{overfit}$ 

- Run each of your X through the function with the y target vector. 
- Recall that the output of your function has the true values of y in one column and then predicted values of y for the different train-test splits in other columns

In [16]:
yhats_full = predict_from_samples(linreg, X, y)
yhats_small = predict_from_samples(linreg, X_small, y)
yhats_over = predict_from_samples(linreg, X_overfit, y)

print(X.shape, X_small.shape, X_overfit.shape)
print(yhats_full.shape, yhats_small.shape, yhats_over.shape)

(777, 17) (777, 1) (777, 833)
(777, 101) (777, 101) (777, 101)


In [17]:
yhats_full.head(5)

,ytrue,sample1,sample2,sample3,sample4,sample5,sample6,sample7,sample8,sample9,...,sample91,sample92,sample93,sample94,sample95,sample96,sample97,sample98,sample99,sample100
0,-0.318252,-0.512825,-0.529464,NaN,NaN,NaN,-0.530716,NaN,-0.450349,-0.490560,...,NaN,-0.660476,NaN,-0.383850,-0.51344,NaN,-0.572368,NaN,NaN,-0.548193
1,-0.551262,NaN,NaN,NaN,-0.255130,NaN,NaN,NaN,-0.325543,-0.062165,...,-0.027846,0.124181,NaN,NaN,NaN,NaN,0.049788,0.108399,NaN,NaN
2,-0.667767,NaN,NaN,NaN,0.097583,NaN,0.162797,NaN,0.082818,NaN,...,0.129454,NaN,NaN,NaN,NaN,NaN,0.086139,0.227121,NaN,NaN
3,-0.376504,0.672275,0.773802,NaN,0.723679,NaN,0.608037,NaN,NaN,NaN,...,NaN,0.655351,NaN,0.704469,NaN,NaN,NaN,NaN,NaN,0.741045
4,-2.939613,NaN,-0.734722,-0.700904,-0.692327,NaN,-0.903917,-0.787925,NaN,NaN,...,NaN,NaN,NaN,-0.668942,NaN,-0.825031,-0.761754,-0.860531,-0.70686,NaN


### Calculate bias and variance 

Below are two functions to calculate bias and variance from the dataframe returned by your function `predict_from_samples`.
You can use them to calculate the bias and variance for the models containing different feature combinations. 

If you have more predictors, variance of prediction should generally go up and bias go down. Likewise, if you have few predictors variance should go down and bias go up.

For a bad model, they both might go up a lot!

In [18]:
def calculate_bias_sq(yhats_df):
    # Take out the true values of y that are in the first column:
    ytrue = yhats_df.iloc[:,0]
    
    # Calculate the mean of the predictions, averaged across the columns.
    # So, all of the predictions for the true y at row 0 would be averaged together
    # and so on for all the rows.
    yhat_means = yhats_df.iloc[:,1:].mean(axis=1)
    
    # Subtract the true value of y from the mean of the predicted values, and square it.
    elementwise_bias_sq = (yhat_means - ytrue)**2
    
    # Take the mean of those squared bias values (across all y)
    mean_bias_sq = np.mean(elementwise_bias_sq)

    return mean_bias_sq

def calculate_variance(yhats_df):
    
    # Calculate the mean of the predicted y's across the columns (mean of yhat for each row)
    yhats_means = yhats_df.iloc[:,1:].mean(axis=1)
    
    # subtract the mean of the yhats from the original yhat values (for each row)
    # and square the result. 
    yhats_devsq = (yhats_df.iloc[:,1:].subtract(yhats_means, axis=0))**2
    
    # Take the mean of the squared deviations from the mean, then 
    # take the mean of those to get the overall variance across the y observations
    yhats_devsq_means = yhats_devsq.mean(axis=1)
    
    return np.mean(yhats_devsq_means)

In [19]:
print("variance full:", calculate_variance(yhats_full))
print("bias full:", calculate_bias_sq(yhats_full))
print("variance small:", calculate_variance(yhats_small))
print("bias small:", calculate_bias_sq(yhats_small))
print("variance over:", calculate_variance(yhats_over))
print("bias over:", calculate_bias_sq(yhats_over))

variance full: 0.009049366891939276
bias full: 0.5741137047263181
variance small: 0.0016063629884423994
bias small: 0.9342491633868237
variance over: 175.81956568079184
bias over: 107.90633141232202


### How does regularization affect bias and variance?

Use lasso and/or ridge regression on your dataset with all the predictor variables. In your function `predict_from_samples` replace `model` with your choice.

In [20]:
from sklearn.linear_model import Lasso, Ridge

lasso = Lasso(alpha=2.0)
ridge = Ridge(alpha=2.1)

In [21]:
yhats_full = predict_from_samples(lasso, X, y)
yhats_small = predict_from_samples(lasso, X_small, y)
yhats_over = predict_from_samples(lasso, X_overfit, y)
print("variance full:", calculate_variance(yhats_full))
print("bias full:", calculate_bias_sq(yhats_full))
print("variance small:", calculate_variance(yhats_small))
print("bias small:", calculate_bias_sq(yhats_small))
print("variance over:", calculate_variance(yhats_over))
print("bias over:", calculate_bias_sq(yhats_over))

variance full: 0.0005702831702019593
bias full: 1.0024269587603027
variance small: 0.0006215609193647028
bias small: 1.0026274719335944
variance over: 0.0005023819421557555
bias over: 1.002151963589761


In [22]:
yhats_full = predict_from_samples(ridge, X, y)
yhats_small = predict_from_samples(ridge, X_small, y)
yhats_over = predict_from_samples(ridge, X_overfit, y)

print("variance full:", calculate_variance(yhats_full))
print("bias full:", calculate_bias_sq(yhats_full))
print("variance small:", calculate_variance(yhats_small))
print("bias small:", calculate_bias_sq(yhats_small))
print("variance over:", calculate_variance(yhats_over))
print("bias over:", calculate_bias_sq(yhats_over))

variance full: 0.0077814844952792065
bias full: 0.5710156876387991
variance small: 0.0014121554619616642
bias small: 0.933413524866396
variance over: 3.325046305901924
bias over: 2.0050184351656157
